# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

## 1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importando as Bibliotecas
import pandas as pd
import requests

In [ ]:
# Carregando o banco de dados
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/EBAC/Limpeza e tratamento de dados /SINASC_RO_2019.csv')
print(df.shape)

# Aplicando o duplicates
df.drop_duplicates().shape


(27028, 69)


(27028, 69)

## 2. Conte o número de valores *missing* por variável.

In [ ]:
# 2) Verificando a quantidade de dados faltantes
df.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [ ]:
# filtrando as colunas que possuem dados faltantes:
df_faltantes = df.loc[:, df.isna().sum() > 0]

# mostrando as colunas com dados faltantes
df_faltantes.columns

Index(['CODESTAB', 'ESTCIVMAE', 'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO',
       'QTDFILMORT', 'GESTACAO', 'GRAVIDEZ', 'PARTO', 'HORANASC', 'SEXO',
       'APGAR1', 'APGAR5', 'RACACOR', 'IDANOMAL', 'CODANOMAL', 'DTRECORIGA',
       'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010', 'SERIESCMAE',
       'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES',
       'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM', 'CONSPRENAT',
       'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO', 'TPNASCASSI',
       'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1', 'munResLat',
       'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [ ]:
# Colunas que possuem dados faltantes e a quantidade de dados:
df_faltantes.isna().sum().sort_values()

munResArea        1
munResLon         1
munResLat         1
munResAlt         1
SEXO              4
TPDOCRESP        14
HORANASC         21
DTNASCMAE        40
PARTO            49
DTDECLARAC       52
TPNASCASSI       61
TPFUNCRESP       67
GRAVIDEZ         79
APGAR1           96
APGAR5          103
CODESTAB        115
ESCMAE2010      249
ESCMAEAGR1      249
TPAPRESENT      265
NATURALMAE      298
CODMUNNATU      298
CODUFNATU       298
ESCMAE          312
ESTCIVMAE       317
IDANOMAL        591
RACACOR         647
RACACORMAE      661
STCESPARTO      747
CONSPRENAT      930
STTRABPART      947
QTDGESTANT     1212
GESTACAO       1232
SEMAGESTAC     1238
TPMETESTIM     1238
QTDFILVIVO     1573
QTDPARTCES     1791
QTDPARTNOR     1879
QTDFILMORT     2098
MESPRENAT      2867
CODOCUPMAE     2907
DTULTMENST    10072
SERIESCMAE    12710
IDADEPAI      19421
CODANOMAL     26814
DTRECORIGA    27028
dtype: int64

In [ ]:
# Percentual dos dados faltantes:
Percentual_df_faltantes = (df_faltantes.isnull().sum() / len(df)) * 100

# Deixando os dados em ordem
Percentual_df_faltantes.sort_values(ascending=False)

DTRECORIGA    100.000000
CODANOMAL      99.208229
IDADEPAI       71.855113
SERIESCMAE     47.025307
DTULTMENST     37.265058
CODOCUPMAE     10.755513
MESPRENAT      10.607518
QTDFILMORT      7.762321
QTDPARTNOR      6.952050
QTDPARTCES      6.626461
QTDFILVIVO      5.819890
SEMAGESTAC      4.580435
TPMETESTIM      4.580435
GESTACAO        4.558236
QTDGESTANT      4.484239
STTRABPART      3.503774
CONSPRENAT      3.440876
STCESPARTO      2.763801
RACACORMAE      2.445612
RACACOR         2.393814
IDANOMAL        2.186621
ESTCIVMAE       1.172858
ESCMAE          1.154358
CODUFNATU       1.102560
NATURALMAE      1.102560
CODMUNNATU      1.102560
TPAPRESENT      0.980465
ESCMAE2010      0.921267
ESCMAEAGR1      0.921267
CODESTAB        0.425485
APGAR5          0.381086
APGAR1          0.355187
GRAVIDEZ        0.292289
TPFUNCRESP      0.247891
TPNASCASSI      0.225692
DTDECLARAC      0.192393
PARTO           0.181293
DTNASCMAE       0.147995
HORANASC        0.077697
TPDOCRESP       0.051798


In [ ]:
# Filtrando os valores maior percentual de dados faltantes:
df_percentual_maior = Percentual_df_faltantes[Percentual_df_faltantes > 15]

# Exibindo as colunas com maiores dados faltantes
df_percentual_maior.sort_values()

DTULTMENST     37.265058
SERIESCMAE     47.025307
IDADEPAI       71.855113
CODANOMAL      99.208229
DTRECORIGA    100.000000
dtype: float64

## 3. No item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.

In [ ]:
# 3) seu código aqui
df_filtrados = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                  'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

# Colunas com dados mais relevantes:
df_filtrados.head()


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [ ]:
# filtrando os dados faltantes
df_filtrados.isna().sum().sort_values()

LOCNASC          0
IDADEMAE         0
CONSULTAS        0
GRAVIDEZ        79
APGAR5         103
ESCMAE         312
ESTCIVMAE      317
GESTACAO      1232
QTDFILVIVO    1573
dtype: int64

In [ ]:
# Filtrando as colunas com valores faltantes
df_faltantes_maior_0 = df_filtrados.loc[:, df_filtrados.isna().sum() > 0]

df_faltantes_maior_0.columns

Index(['ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'APGAR5'], dtype='object')

## 4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.

In [ ]:
# 4) seu código aqui
# O comando .shape() verifica a quantidade de linhas na coluna apgar5
df['APGAR5'].shape

(27028,)

In [ ]:
#  Somatório da quantidade de linhas missing
df['APGAR5'].isna().sum()

103

In [ ]:
#  com o comando dropna() retiramos os dados faltantes
df_apgar5 = df['APGAR5'].dropna()
df_apgar5

0        10.0
1         9.0
2        10.0
3        10.0
4        10.0
         ... 
27023     9.0
27024     9.0
27025    10.0
27026     9.0
27027     8.0
Name: APGAR5, Length: 26925, dtype: float64

## 5. Observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>

In [ ]:
# 5) seu código aqui

# Filtrando os dados faltantes na variável "ESTCIVMAE"
df['ESTCIVMAE'].value_counts().sort_index()

1.0    8952
2.0    9924
3.0      57
4.0     453
5.0    7180
9.0     145
Name: ESTCIVMAE, dtype: int64

In [ ]:
# Filtrando os dados faltantes na variável "CONSULTAS"
df['CONSULTAS'].value_counts().sort_index()

1      986
2     1379
3     5988
4    18624
9       51
Name: CONSULTAS, dtype: int64

In [ ]:
# Removendo os valores ignorados das variáveis
df_ign = df.loc[(df['CONSULTAS'] != 9) & (df['ESTCIVMAE'] != 9)]
df_ign['CONSULTAS'].value_counts().sort_index()

1      980
2     1374
3     5955
4    18523
Name: CONSULTAS, dtype: int64

## 6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.

In [ ]:
# 6) Seu código aqui

# Somatório dos valores faltantes
df['QTDFILVIVO'].isna().sum()

1573

In [ ]:
# Usamos a função .fillna para atribuir (0) nos dados faltantes
df_filhos_vivos= df['QTDFILVIVO'].fillna(0)

# Visualizando os dados
df_filhos_vivos

0        0.0
1        1.0
2        2.0
3        0.0
4        1.0
        ... 
27023    1.0
27024    0.0
27025    0.0
27026    1.0
27027    1.0
Name: QTDFILVIVO, Length: 27028, dtype: float64

In [ ]:
# Filtragem quantidade de filhos vivos
df_filhos_vivos.value_counts()

0.0     10899
1.0      9390
2.0      4474
3.0      1370
4.0       484
5.0       214
6.0       112
7.0        42
8.0        23
9.0         8
10.0        6
12.0        3
11.0        1
30.0        1
14.0        1
Name: QTDFILVIVO, dtype: int64

## 7. Das restantes, decida que valor te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

In [ ]:
# 7) seu código aqui

# Colunas com dados faltantes
df_faltantes_maior_0 .columns

Index(['ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'APGAR5'], dtype='object')

In [ ]:
df['GRAVIDEZ'].value_counts()

Única              26452
Dupla                492
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [ ]:
# Somatório dos dados faltantes na coluna Gravidez
df['GRAVIDEZ'].isna().sum()

79

In [ ]:
# Aplicando .dropna vamos elimanar os dados inexistente da coluna "GRAVIDEZ"
df_gravidez = df['GRAVIDEZ'].dropna()
df_gravidez.value_counts()

Única              26452
Dupla                492
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [ ]:
# Eliminando os valores nulos e iguais a 9
df_estado_civil= df.loc[(df['ESTCIVMAE'] != 9) & (df['ESTCIVMAE'].notna())]
df_estado_civil['ESTCIVMAE'].value_counts()

2.0    9924
1.0    8952
5.0    7180
4.0     453
3.0      57
Name: ESTCIVMAE, dtype: int64

## 8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:

- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [ ]:
# verificando os dados
df['APGAR5'].value_counts().sort_values()

3.0        15
0.0        17
2.0        18
4.0        23
1.0        24
5.0        45
6.0        79
7.0       241
8.0      1994
10.0     8393
9.0     16076
Name: APGAR5, dtype: int64

In [ ]:
# Categorizando dos dados
df.loc[df['APGAR5'] >= 8, 'apgar5_novo'] = 'normal'
df.loc[(df['APGAR5'] >=6) & (df['APGAR5'] <= 7),
           'apgar5_novo'] = 'asfixia leve'
df.loc[(df['APGAR5'] >= 4) & (df['APGAR5'] <= 5),
           'apgar5_novo'] = 'asfixia moderada'
df.loc[(df['APGAR5'] <= 3), 'apgar5_novo'] = 'asfixia severa'

In [ ]:
# mostrando a frequência
df['apgar5_novo'].value_counts()

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar5_novo, dtype: int64

## 9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [ ]:
# 9) seu código aqui

# transformando os valores em snake case
df['apgar5_novo'] = df['apgar5_novo'].str.replace(' ', '_').str.lower()

In [ ]:
# Verificando a frequencia
df['apgar5_novo'].value_counts()

normal              26463
asfixia_leve          320
asfixia_severa         74
asfixia_moderada       68
Name: apgar5_novo, dtype: int64